In [1]:
using Test, pulse_input_DDM, LinearAlgebra, Flatten, Parameters

┌ Info: Recompiling stale cache file /usr/people/briandd/.julia/compiled/v1.2/pulse_input_DDM/m8WoA.ji for pulse_input_DDM [c2542da2-1f7d-11e9-2506-87691095ebb2]
└ @ Base loading.jl:1240


In [2]:
n = 53

53

In [3]:
## Choice model
θ = θchoice(θz=θz(σ2_i = 0.5, B = 15., λ = -0.5, σ2_a = 50., σ2_s = 1.5,
    ϕ = 0.8, τ_ϕ = 0.05),
    bias=1., lapse=0.05)

θ, data = synthetic_data(;θ=θ, ntrials=10, rng=1)
model_gen = choiceDDM(θ, data)

choiceDDM{θchoice{θz{Float64},Float64},Array{choicedata{pulse_input_DDM.choiceinputs{pulse_input_DDM.clicks,pulse_input_DDM.binned_clicks}},1}}
  θ: θchoice{θz{Float64},Float64}
  data: Array{choicedata{pulse_input_DDM.choiceinputs{pulse_input_DDM.clicks,pulse_input_DDM.binned_clicks}}}((10,))


In [4]:
choices = getfield.(data, :choice);

In [5]:
@test all(choices .== vcat(falses(9), trues(1)))

Test Passed

In [6]:
@time @test round(loglikelihood(model_gen), digits=2) ≈ -3.72

  1.652521 seconds (4.17 M allocations: 226.213 MiB, 8.91% gc time)


Test Passed

In [7]:
@time @test round(loglikelihood(θ, data), digits=2) ≈ -3.72

  0.074668 seconds (9.98 k allocations: 15.813 MiB, 18.96% gc time)


Test Passed

In [8]:
@test round(θ(data), digits=2) ≈ -3.72

Test Passed

In [ ]:
@test round(norm(gradient(model_gen)), digits=2) ≈ 14.32

In [10]:
options = choiceoptions(fit = vcat(trues(9)),
    lb = vcat([0., 8., -5., 0., 0., 0.01, 0.005], [-30, 0.]),
    ub = vcat([2., 30., 5., 100., 2.5, 1.2, 1.], [30, 1.]),
    x0 = vcat([0.1, 15., -0.1, 20., 0.5, 0.8, 0.008], [0.,0.01]))

model, = optimize(data, options; iterations=5, outer_iterations=1);
@test round(norm(Flatten.flatten(model.θ)), digits=2) ≈ 25.05

Fminbox
-------
Initial mu = 0.000251462

Fminbox iteration 1
-------------------
Calling inner optimizer with mu = 0.000251462

(numbers below include barrier contribution)
Iter     Function value   Gradient norm 
     0     7.774665e+00     9.860045e+01
 * time: 7.295608520507812e-5

Exiting inner optimizer with x = [0.10908907449676107, 14.998922611692793, -0.119808301908876, 19.99841299411124, 0.5668629214289671, 1.1815218500705917, 0.2572173860091957, 0.29433081444119363, 0.652638765006292]
Current distance to box: 0.0184781
Decreasing barrier term μ.

optimization complete. converged: false 



Test Passed

In [11]:
H = Hessian(model)
@test round(norm(H), digits=2) ≈ 92.6

Test Passed

In [12]:
CI, HPSD = CIs(H)
@test round(norm(CI), digits=2) ≈ 1471.54

┌ Warning: Hessian is not positive definite. Approximated by closest PSD matrix.
│             ||ϵ||/||H|| is 0.016142554603615196
└ @ pulse_input_DDM /usr/people/briandd/Projects/pulse_input_DDM/src/base_model.jl:19


Test Passed

In [13]:
## Neural model
f, ncells, ntrials, nparams = "Sigmoid", [1,2], [10,5], 4

θ = θneural(θz = θz(σ2_i = 0.5, B = 15., λ = -0.5, σ2_a = 10., σ2_s = 1.2,
    ϕ = 0.6, τ_ϕ =  0.02),
    θy=[[Sigmoid() for n in 1:N] for N in ncells], ncells=ncells,
    nparams=nparams, f=f);

In [14]:
data, = synthetic_data(θ, ntrials);
model_gen = neuralDDM(θ, data);

In [18]:
spikes = map(x-> sum.(x), getfield.(vcat(data...), :spikes))

15-element Array{Array{Int64,1},1}:
 [2]     
 [13]    
 [4]     
 [1]     
 [4]     
 [9]     
 [21]    
 [9]     
 [7]     
 [23]    
 [11, 10]
 [3, 6]  
 [7, 6]  
 [10, 10]
 [10, 9] 

In [16]:
@test all(spikes .== [[3], [14], [6], [1], [5], [9], [20], [9], [5], [21], [11, 10], [5, 8], [7, 5], [10, 11], [11, 9]])

Test Failed at In[16]:1
  Expression: all(spikes .== [[3], [14], [6], [1], [5], [9], [20], [9], [5], [21], [11, 10], [5, 8], [7, 5], [10, 11], [11, 9]])


Test.FallbackTestSetException: There was an error during testing

In [19]:
@test round(loglikelihood(model_gen), digits=2) ≈ -529.86

Test Passed

In [20]:
@test round(norm(gradient(model_gen)), digits=2) ≈ 52.18

Test Failed at In[20]:1
  Expression: round(norm(gradient(model_gen)), digits=2) ≈ 63.56
   Evaluated: 63.51 ≈ 63.56


Test.FallbackTestSetException: There was an error during testing

In [21]:
x = pulse_input_DDM.flatten(θ)
@test round(loglikelihood(x, data, θ), digits=2) ≈ -529.86

Test Passed

In [22]:
θy0 = vcat(vcat(θy.(data, f)...)...)
@test round(norm(θy0), digits=2) ≈ 31.16

Test Failed at In[22]:2
  Expression: round(norm(θy0), digits=2) ≈ 53.56
   Evaluated: 38.81 ≈ 53.56


Test.FallbackTestSetException: There was an error during testing

In [23]:
#deterministic model
options0 = Sigmoid_options_noiseless(ncells=ncells,
    fit=vcat(falses(dimz), trues(sum(ncells)*nparams)),
    x0=vcat([0., 30., 0. + eps(), 0., 0., 1. - eps(), 0.008], θy0))

Sigmoid_options_noiseless
  ncells: Array{Int64}((2,)) [1, 2]
  nparams: Int64 4
  f: String "Sigmoid"
  fit: Array{Bool}((19,)) Bool[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
  lb: Array{Float64}((19,)) [0.0, 8.0, -5.0, 0.0, 0.0, 0.01, 0.005, -100.0, 0.0, -10.0, -10.0, -100.0, 0.0, -10.0, -10.0, -100.0, 0.0, -10.0, -10.0]
  ub: Array{Float64}((19,)) [Inf, Inf, 10.0, Inf, Inf, 1.2, 1.0, 100.0, 100.0, 10.0, 10.0, 100.0, 100.0, 10.0, 10.0, 100.0, 100.0, 10.0, 10.0]
  x0: Array{Float64}((19,)) [0.0, 30.0, 2.220446049250313e-16, 0.0, 0.0, 0.9999999999999998, 0.008, -11.823240890025499, 10.9789569121723, 0.0012275100524168838, 0.0, -14.192838387217776, 21.261707067327677, 0.008475145450118802, 0.0, -7.880146395368877, 23.02894451372203, 0.0058347955302947676, 0.0]


In [24]:
θ0 = θneural_noiseless(options0.x0, ncells, nparams, f)
model0 = neuralDDM(θ0, data)

neuralDDM{θneural_noiseless{θz{Float64},Array{Array{Sigmoid{Float64},1},1}},Array{Array{neuraldata,1},1}}
  θ: θneural_noiseless{θz{Float64},Array{Array{Sigmoid{Float64},1},1}}
  data: Array{Array{neuraldata,1}}((2,))


In [25]:
@test round(loglikelihood(model0), digits=2) ≈ -1399.68

Test Failed at In[25]:1
  Expression: round(loglikelihood(model0), digits=2) ≈ -533.09
   Evaluated: -1495.56 ≈ -533.09


Test.FallbackTestSetException: There was an error during testing

In [20]:
x0 = pulse_input_DDM.flatten(θ0)
@unpack ncells, nparams, f = θ0

θneural_noiseless{θz{Float64},Array{Array{Sigmoid{Float64},1},1}}
  θz: θz{Float64}
  θy: Array{Array{Sigmoid{Float64},1}}((2,))
  ncells: Array{Int64}((2,)) [1, 2]
  nparams: Int64 4
  f: String "Sigmoid"


In [23]:
@test round(loglikelihood(x0, data, θ0), digits=2) ≈ -1399.68

Test Passed

In [25]:
model, = optimize(data, options0; iterations=2, outer_iterations=1)
@test round(norm(pulse_input_DDM.flatten(model.θ)), digits=2) ≈ 87.51

Fminbox
-------
Initial mu = 1.59053

Fminbox iteration 1
-------------------
Calling inner optimizer with mu = 1.59053

(numbers below include barrier contribution)
Iter     Function value   Gradient norm 
     0     1.281226e+03     9.422631e+02
 * time: 3.409385681152344e-5

Exiting inner optimizer with x = [52.69070630256158, 43.434427853058395, -2.8625905261748006, 5.0336085210607395, 17.047935166694103, 33.8056973114395, 5.1937297903733795, 3.97561896305667, -5.690347702664896, 23.203791176447094, 0.4792342779693217, 3.6141666197326594]
Current distance to box: 4.80627
Decreasing barrier term μ.



Test Passed

In [27]:
@test round(norm(gradient(model)), digits=2) ≈ 6.82

Test Passed

In [33]:
x0 = vcat([0.1, 15., -0.1, 20., 0.5, 0.8, 0.008], pulse_input_DDM.flatten(model.θ)[dimz+1:end])

19-element Array{Float64,1}:
  0.1               
 15.0               
 -0.1               
 20.0               
  0.5               
  0.8               
  0.008             
 52.69070630256158  
 43.434427853058395 
 -2.8625905261748006
  5.0336085210607395
 17.047935166694103 
 33.8056973114395   
  5.1937297903733795
  3.97561896305667  
 -5.690347702664896 
 23.203791176447094 
  0.4792342779693217
  3.6141666197326594

In [35]:
options = Sigmoid_options(ncells=ncells, x0=x0)

model, = optimize(data, options; iterations=2, outer_iterations=1)
@test round(norm(pulse_input_DDM.flatten(model.θ)), digits=2) ≈ 85.73

Fminbox
-------
Initial mu = 0.00248823

Fminbox iteration 1
-------------------
Calling inner optimizer with mu = 0.00248823

(numbers below include barrier contribution)
Iter     Function value   Gradient norm 
     0     7.682817e+02     7.356741e+02
 * time: 8.702278137207031e-5

Exiting inner optimizer with x = [1.0518193782012302, 15.036691202576568, 0.2797663017620923, 20.040066830076672, 0.5926527356050355, 0.018393557393958426, 0.7082382774236562, 52.42451557297762, 43.404607103709104, -3.366376053911049, 4.683811140044606, 16.99575905575336, 33.79209919106686, 4.917694861086083, 3.9073304775861413, -5.6894928464434855, 23.202985062384148, 0.2809125301584484, 3.6334904947467415]
Current distance to box: 0.00839356
Decreasing barrier term μ.



Test Passed

In [39]:
H = Hessian(model, n; chuck_size=4)
@test round(norm(H), digits=2) ≈ 22.83

Test Passed

In [41]:
CI, HPSD = CIs(H)
@test round(norm(CI), digits=2) ≈ 303.02

┌ Warning: Hessian is not positive definite. Approximated by closest PSD matrix.
│             ||ϵ||/||H|| is 1.782416929642061
└ @ pulse_input_DDM /usr/people/briandd/Projects/pulse_input_DDM/src/base_model.jl:19


Test Passed